# Optical flow Gaussian image denoising

In [ ]:
#use_OpenCV = False
use_OpenCV = True
local_debug = True

In [ ]:
import logging
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
#from ipywidgets import *
import cv2
import time
#import kernels
import mrcfile
from collections import namedtuple

In [ ]:
import logging
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)

In [ ]:
if local_debug:
    !ln -sf ../../information_theory/src/information_theory/ .
else:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
import information_theory  # pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
if local_debug:
    !ln -sf ../../motion_estimation/src/motion_estimation/ .
else:
    !pip install "motion_estimation @ git+https://github.com/vicente-gonzalez-ruiz/motion_estimation"
if use_OpenCV:
    from motion_estimation._2D.farneback_OpenCV import Estimator_in_CPU as Estimator
    from motion_estimation._2D.project import project
else:
    from motion_estimation._1D.farneback_python import Estimator
    from motion_estimation._1D.project import project

In [ ]:
Args = namedtuple("args", "input")
args = Args("FoilHole.mrc")

In [ ]:
%%bash
rm {args.input}
OUTPUT_FILENAME={args.input}
if test ! -f $OUTPUT_FILENAME ; then
    FILEID="11lmIglqAgIgTRS3YZZccDM-ma2bI-CXr"
    wget --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O $OUTPUT_FILENAME 2> /dev/null
fi

In [ ]:
!ls -l {args.input}

In [ ]:
stack_MRC = mrcfile.open(args.input)
noisy = stack_MRC.data

In [ ]:
np.max(noisy)

In [ ]:
np.min(noisy)

In [ ]:
noisy = (255*(noisy - np.min(noisy))/(np.max(noisy) - np.min(noisy))).astype(np.uint8)

In [ ]:
plt.imshow(noisy, cmap="gray")

In [ ]:
padded_noisy = np.zeros(shape=(noisy.shape[0]+32, noisy.shape[1]+32), dtype=np.uint8)
padded_noisy[16:-16, 16:-16] = noisy
noisy = padded_noisy

In [ ]:
plt.imshow(noisy, cmap="gray")

In [ ]:
def get_gaussian_kernel(sigma=1):
    number_of_coeffs = 3
    number_of_zeros = 0
    while number_of_zeros < 2 :
        delta = np.zeros(number_of_coeffs)
        delta[delta.size//2] = 1
        coeffs = scipy.ndimage.gaussian_filter1d(delta, sigma=sigma)
        number_of_zeros = coeffs.size - np.count_nonzero(coeffs)
        number_of_coeffs += 1
    return coeffs[1:-1]

#sigma = np.array([1.0, 1.0])
sigma = np.array([1.5, 1.5])
#sigma = np.array([2.5, 2.5])
#sigma = np.array([2.0, 2.0])
#sigma = np.array([3.0, 3.0])
kernel = [None]*2
kernel[0] = get_gaussian_kernel(sigma[0])
kernel[1] = get_gaussian_kernel(sigma[1])
print(np.sum(kernel[0]))
plt.plot(kernel[0])
plt.show()

In [ ]:
if use_OpenCV:
    import denoising.image.OF_gaussian__OpenCV as denoising
else:
    import denoising.image.OF_gaussian as denoising

In [ ]:
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)
if use_OpenCV:
    denoiser = denoising.Monochrome_Denoising(logger, N_poly=9, window_side=5)
else:
    denoiser = denoising.Monochrome_Denoising(logger, N_poly=101, window_length=101)

In [ ]:
if use_OpenCV:
    denoised = denoiser.filter(noisy, kernel)
else:
    denoised = denoiser.filter(noisy.astype(np.float64), kernel)

In [ ]:
np.min(denoised)

In [ ]:
np.max(denoised)

In [ ]:
denoised = np.clip(denoised, a_min=0, a_max=255)

In [ ]:
denoised = np.round(denoised).astype(np.uint8)

In [ ]:
np.min(denoised)

In [ ]:
np.max(denoised)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy, cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised, cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy, denoised)})")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[200:400,800:1000], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[200:400,800:1000], cmap="gray")
axs[1].set_title(f"Denoised")
fig.tight_layout()
plt.show()

In [ ]:
np.mean(noisy)

In [ ]:
np.mean(denoised)

In [ ]:
from skimage.metrics import structural_similarity as ssim
from scipy import stats

In [ ]:
diff = (noisy - denoised).astype(np.uint8)

In [ ]:
plt.imshow(diff, cmap="gray")

In [ ]:
_, N = ssim(noisy, diff, full=True)

In [ ]:
plt.imshow(N, cmap="gray")

In [ ]:
_, P = ssim(noisy, denoised.astype(np.uint8), full=True)

In [ ]:
plt.imshow(P, cmap="gray")

In [ ]:
quality, _ = stats.pearsonr(N.flatten(), P.flatten())

In [ ]:
quality